In [1]:
import pandas as pd
import numpy as np
 
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter

from utils_data import Vocabulary, Vectorizer, clean_words
from training import train, evaluate


%matplotlib inline
%load_ext autoreload
%autoreload 2

In [38]:
from datasets import load_dataset

# data_en = load_dataset('head_qa', 'en')
data_es = load_dataset('head_qa', 'es' )
data_es

Reusing dataset head_qa (C:\Users\tec005m\.cache\huggingface\datasets\head_qa\es\1.1.0\473dc5357942a3ff52963bd73cad0d167bd1bbc1ca5ca0732ee7372b480dd735)


DatasetDict({
    train: Dataset({
        features: ['name', 'year', 'category', 'qid', 'qtext', 'ra', 'image', 'answers'],
        num_rows: 2657
    })
    test: Dataset({
        features: ['name', 'year', 'category', 'qid', 'qtext', 'ra', 'image', 'answers'],
        num_rows: 2742
    })
    validation: Dataset({
        features: ['name', 'year', 'category', 'qid', 'qtext', 'ra', 'image', 'answers'],
        num_rows: 1366
    })
})

In [9]:
training, validation, testing = data_es['train'], data_es['validation'], data_es['test']

In [10]:
for d in training:
    print(d)
    break;

{'answers': [{'aid': 1, 'atext': 'Son de tipo todo o nada.'}, {'aid': 2, 'atext': 'Son hiperpolarizantes.'}, {'aid': 3, 'atext': 'Se pueden sumar.'}, {'aid': 4, 'atext': 'Se propagan a largas distancias.'}, {'aid': 5, 'atext': 'Presentan un periodo refractario.'}], 'category': 'biology', 'image': '', 'name': 'Cuaderno_2013_1_B', 'qid': 1, 'qtext': 'Los potenciales postsinápticos excitadores:', 'ra': 3, 'year': '2013'}


In [5]:
#!python -m spacy download es_core_news_sm

In [6]:
import spacy
nlp = spacy.load("es_core_news_sm")

vocabulary = Vocabulary()

In [21]:
def parse_training(training):
    train = []       
    for sample in training:
        training_sample = {}
        qtext = sample['qtext']
        answers = sample['answers']        
        q = nlp(qtext)
        tok_qtext = [token.text for token in q]
        right_answer = sample['ra']        
        for answer in answers:
            aid, atext = answer['aid'], answer['atext']
            a = nlp(atext)
            tok_atext = [token.text for token in a]
            instance_x = tok_qtext + ['SEP'] + tok_atext    
            instance_y = 1 if right_answer == aid else 0
            training_sample['question'] = qtext
            training_sample['answer'] = atext
            training_sample['label'] = instance_y
            training_sample['sample_tok'] = instance_x
            training_sample['category'] = sample['category']
            train.append(training_sample)
    return train

In [22]:
# def evaluate(dataset, vocabulary, )

In [25]:
instances = parse_training(training)

In [35]:
instances[200]

{'question': 'La presión alveolar supera a la atmosférica:',
 'answer': 'Al finalizar la inspiración.',
 'label': 0,
 'sample_tok': ['La',
  'presión',
  'alveolar',
  'supera',
  'a',
  'la',
  'atmosférica',
  ':',
  'SEP',
  'Al',
  'finalizar',
  'la',
  'inspiración',
  '.'],
 'category': 'biology'}

In [39]:
vectorizer = Vectorizer.vectorize_training(instances)

In [43]:
vectorizer.vocab2index

{'UNK': 0,
 'Los': 1,
 'potenciales': 2,
 'postsinápticos': 3,
 'excitadores': 4,
 ':': 5,
 'SEP': 6,
 'Presentan': 7,
 'un': 8,
 'periodo': 9,
 'refractario': 10,
 '.': 11,
 'Placa': 12,
 'motora': 13,
 'es': 14,
 'la': 15,
 'unión': 16,
 'entre': 17,
 'neurona': 18,
 'y': 19,
 'el': 20,
 'Tendón': 21,
 'NO': 22,
 'generan': 23,
 'de': 24,
 'acción': 25,
 'Neuronas': 26,
 'ganglionares': 27,
 'retina': 28,
 'En': 29,
 'iniciación': 30,
 'los': 31,
 'movimientos': 32,
 'voluntarios': 33,
 'primera': 34,
 'zona': 35,
 'que': 36,
 'se': 37,
 'activa': 38,
 'Ganglios': 39,
 'basales': 40,
 'corpúsculos': 41,
 'Pacini': 42,
 'Son': 43,
 'termorreceptores': 44,
 'El': 45,
 'color': 46,
 'ojos': 47,
 'depende': 48,
 'del': 49,
 'pigmento': 50,
 'presente': 51,
 'en': 52,
 'Retina': 53,
 'conos': 54,
 'Sinaptan': 55,
 'sólo': 56,
 'sobre': 57,
 'bipolares': 58,
 'centro': 59,
 '“': 60,
 'on': 61,
 '”': 62,
 'capa': 63,
 'más': 64,
 'próxima': 65,
 'al': 66,
 'epitelio': 67,
 'pigmentario': 68

In [ ]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, x_size, n_classes): 
        super(LogisticRegression, self).__init__()             
        self.linear = nn.Linear(x_size, n_classes)
        
    def forward(self, x):
        return self.linear(x)